In [55]:
from PIL import Image

img = Image.new( 'RGB', (500,250), "white") # create a new white image
pixels = img.load()

In [61]:
def log_prod(code1, code2):
    p = 0
    for i in range(4):
        p += code1[i] & code2[i]

    return p


def is_visible(bar, rect):
    """Видимость - 0 = невидимый
                   1 = видимый
                   2 = частично видимый"""
    # вычисление кодов концевых точек отрезка
    s1 = sum(get_code(bar[0], rect))
    s2 = sum(get_code(bar[1], rect))

    # предположим, что отрезок частично видим
    vis = 2

    # проверка полной видимости отрезка
    if not s1 and not s2:
        vis = 1
    else:
        # проверка тривиальной невидимости отрезка
        l = log_prod(get_code(bar[0], rect), get_code(bar[1], rect))
        if l != 0:
            vis = 0

    return vis


def cohen_sutherland(bar, rect, win):
    # инициализация флага
    flag = 1 # общего положения
    t = 1

    # проверка вертикальности и горизонтальности отрезка
    if bar[1][0] - bar[0][0] == 0:
        flag = -1   # вертикальный отрезок
    else:
        # вычисление наклона
        t = (bar[1][1] - bar[0][1]) / (bar[1][0] - bar[0][0])
        if t == 0:
            flag = 0   # горизонтальный

    # для каждой стороны окна
    for i in range(4):
        vis = is_visible(bar, rect)
        if vis == 1:
            win.scene.addLine(bar[0][0], bar[0][1], bar[1][0], bar[1][1], win.pen)
            return
        elif not vis:
            return

        # проверка пересечения отрезка и стороны окна
        code1 = get_code(bar[0], rect)
        code2 = get_code(bar[1], rect)

        if code1[i] == code2[i]:
            continue

        # проверка нахождения Р1 вне окна; если Р1 внутри окна, то Р2 и Р1 поменять местами
        if not code1[i]:
            bar[0], bar[1] = bar[1], bar[0]

        # поиск пересечений отрезка со сторонами окна
        # контроль вертикальности отрезка
        if flag != -1:
            if i < 2:
                bar[0][1] = t * (rect[i] - bar[0][0]) + bar[0][1]
                bar[0][0] = rect[i]
                continue
            else:
                bar[0][0] = (1 / t) * (rect[i] - bar[0][1]) + bar[0][0]

        bar[0][1] = rect[i]
    win.scene.addLine(bar[0][0], bar[0][1], bar[1][0], bar[1][1], win.pen)

In [ ]:
def isConvex(edges):
    flag = 1

    # начальные вершины
    vo = edges[0]  # iая вершина
    vi = edges[1]  # i+1 вершина
    vn = edges[2]  # i+2 вершина и все остальные

    # векторное произведение двух векторов
    x1 = vi.x() - vo.x()
    y1 = vi.y() - vo.y()

    x2 = vn.x() - vi.x()
    y2 = vn.y() - vi.y()

    # определяем знак ординаты
    r = x1 * y2 - x2 * y1
    prev = sign(r)

    for i in range(2, len(edges) - 1):
        if not flag:
            break
        vo = edges[i - 1]
        vi = edges[i]
        vn = edges[i + 1]

        # векторное произведение двух векторов
        x1 = vi.x() - vo.x()
        y1 = vi.y() - vo.y()

        x2 = vn.x() - vi.x()
        y2 = vn.y() - vi.y()

        r = x1 * y2 - x2 * y1
        curr = sign(r)

        # если знак предыдущей координаты не совпадает, то возможно многоугольник невыпуклый
        if curr != prev:
            flag = 0
        prev = curr

    # не забываем проверить последнюю с первой вершины
    vo = edges[len(edges) - 1]
    vi = edges[0]
    vn = edges[1]

    # векторное произведение двух векторов
    x1 = vi.x() - vo.x()
    y1 = vi.y() - vo.y()

    x2 = vn.x() - vi.x()
    y2 = vn.y() - vi.y()

    r = x1 * y2 - x2 * y1
    curr = sign(r)
    if curr != prev:
        flag = 0

    return flag * curr


def scalar(v1, v2):
    return v1.x() * v2.x() + v1.y() * v2.y()


def cyrus_beck(r, edges, n, scene, p):
    # инициализируем пределы значений параметра, предполагая, что весь отрезок полностью видимый
    # максимизируем t нижнее и t верхнее, исходя из того что 0 <= t <= 1
    tb = 0
    te = 1

    # вычисляем директрису(определяет направление/ориентацию отрезка) D= p1 - p2
    D = QPointF()
    D.setX(r[1][0] - r[0][0])
    D.setY(r[1][1] - r[0][1])

    # главный цикл по сторонам отсекателя
    for i in range(len(edges)):
        # вычисляем wi, D * ni, wi * n
        # весовой множитель удаленности гранничной точки от р1(берем граничную точку равной вершине)
        W = QPointF()
        W.setX(r[0][0] - edges[i].x())
        W.setY(r[0][1] - edges[i].y())

        # определяем нормаль
        N = QPointF()
        if i == len(edges) - 1:
            N.setX(-n * (edges[0].y() - edges[i].y()))
            N.setY(n * (edges[0].x() - edges[i].x()))
        else:
            N.setX(-n * (edges[i + 1].y() - edges[i].y()))
            N.setY(n * (edges[i + 1].x() - edges[i].x()))
        # определяем скалярные произведения
        Dscalar = scalar(D, N)
        Wscalar = scalar(W, N)

        if Dscalar == 0:
            # если отрезок параллелен ребру отсекателю
            if Wscalar < 0:
                # виден ли?
                return
        else:
            # отрезок невырожден, определяем t
            t = - Wscalar / Dscalar
            # поиск верхнего и нижнего предела t

            if Dscalar > 0:
                # поиск нижнего предела
                # верно ли, что t <= 1
                if t > 1:
                    return
                else:
                    tb = max(tb, t)
            elif Dscalar < 0:
                # поиск верхнего предела
                # верно ли, что t >= 0
                if t < 0:
                    return
                else:
                    te = min(te, t)

        # проверка фактической видимости отрезка
    if tb <= te:
        scene.addLine(r[0][0] + (r[1][0] - r[0][0]) * te,
                      r[0][1] + (r[1][1] - r[0][1]) * te,
                      r[0][0] + (r[1][0] - r[0][0]) * tb,
                      r[0][1] + (r[1][1] - r[0][1]) * tb, p)